# Day 7: Interview Review & Practice Problems

## Week 12 - Backtesting & Validation

### 🎯 Learning Objectives
- Review all Week 12 concepts
- Practice interview-style questions
- Solve timed coding challenges
- Build confidence for quant interviews

### ⏱️ Time Allocation
- Concept review: 30 min
- Coding challenges: 90 min
- Interview Q&A: 60 min
- Mock interview: 30 min

---

**Author**: ML Quant Finance Mastery  
**Difficulty**: Interview Level  
**Prerequisites**: Week 12 Day 1-6

## 1. Week 12 Concept Summary

| Day | Topic | Key Concepts |
|-----|-------|--------------|
| 1 | Performance Metrics | Sharpe, Sortino, Max Drawdown, Calmar |
| 2 | Transaction Costs | Commission, spread, slippage, market impact |
| 3 | Walk-Forward | Time series CV, expanding vs rolling, embargo |
| 4 | Best Practices | Point-in-time, survivorship bias, sanity checks |
| 5 | Common Pitfalls | Lookahead, overfitting, multiple testing |
| 6 | Full Pipeline | End-to-end backtest framework |

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# Generate data for exercises
n = 1000
returns_data = np.random.randn(n) * 0.01 + 0.0002
prices_data = 100 * np.cumprod(1 + returns_data)

print("📊 Practice data loaded")
print(f"   {n} days of returns")
print(f"   Mean daily return: {np.mean(returns_data):.4%}")

## 2. Interview Questions - Theory

### Q1: Explain the Sharpe Ratio and its limitations.

**Answer Framework:**

In [ ]:
# Sharpe Ratio demonstration and limitations

def sharpe_ratio(returns, rf=0):
    """Calculate annualized Sharpe Ratio"""
    excess = returns - rf
    return np.mean(excess) / np.std(returns) * np.sqrt(252)

# Standard calculation
sharpe = sharpe_ratio(returns_data)
print(f"Sharpe Ratio: {sharpe:.2f}")

# Limitations:
print("\n📊 SHARPE RATIO LIMITATIONS:")
print("1. Assumes normal distribution (returns have fat tails)")
print("2. Penalizes upside and downside volatility equally")
print("3. Sensitive to measurement period")
print("4. Can be manipulated (sell OTM puts)")

# Demonstrate sensitivity
periods = [63, 126, 252, 500]
print("\nPeriod Sensitivity:")
for p in periods:
    s = sharpe_ratio(returns_data[-p:])
    print(f"   Last {p} days: Sharpe = {s:.2f}")

### Q2: Why does K-Fold CV fail for time series?

**Answer Framework:**

In [ ]:
# Demonstrate lookahead bias in K-Fold

from sklearn.model_selection import KFold, TimeSeriesSplit

# Create features and target
X = np.column_stack([
    np.roll(returns_data, 1),  # lag 1
    np.roll(returns_data, 5),  # lag 5
])[10:]
y = returns_data[10:]

# K-Fold (WRONG)
kfold = KFold(n_splits=5, shuffle=True)
kfold_scores = []
for train_idx, test_idx in kfold.split(X):
    model = Ridge(alpha=1.0)
    model.fit(X[train_idx], y[train_idx])
    score = model.score(X[test_idx], y[test_idx])
    kfold_scores.append(score)

# Time Series (CORRECT)
tscv = TimeSeriesSplit(n_splits=5)
ts_scores = []
for train_idx, test_idx in tscv.split(X):
    model = Ridge(alpha=1.0)
    model.fit(X[train_idx], y[train_idx])
    score = model.score(X[test_idx], y[test_idx])
    ts_scores.append(score)

print("📊 CV COMPARISON:")
print(f"K-Fold (WRONG): R² = {np.mean(kfold_scores):.4f}")
print(f"TimeSeriesSplit (CORRECT): R² = {np.mean(ts_scores):.4f}")
print("\n⚠️ K-Fold uses future data to predict past = LOOKAHEAD BIAS")

### Q3: How do you account for transaction costs in backtests?

**Answer Framework:**

In [ ]:
def backtest_with_costs(signals, returns, cost_bps=10):
    """
    Backtest with transaction costs
    
    Cost model: cost_bps per round-trip trade
    """
    cost = cost_bps / 10000
    
    # Position changes = turnover
    turnover = np.abs(np.diff(signals, prepend=0))
    
    # Gross returns
    gross_returns = signals * returns
    
    # Net returns
    transaction_costs = turnover * cost
    net_returns = gross_returns - transaction_costs
    
    return {
        'gross': gross_returns,
        'net': net_returns,
        'costs': transaction_costs.sum(),
        'turnover': turnover.sum()
    }

# Create momentum signal
momentum = pd.Series(returns_data).rolling(20).mean().values
signal = np.sign(momentum)
signal = np.nan_to_num(signal)

# Compare with different costs
print("📊 TRANSACTION COST IMPACT:")
for cost in [0, 5, 10, 20, 50]:
    result = backtest_with_costs(signal, returns_data, cost)
    sharpe_net = np.mean(result['net']) / np.std(result['net']) * np.sqrt(252)
    print(f"   {cost:2d} bps: Net Sharpe = {sharpe_net:.2f}")

## 3. Coding Challenges

### Challenge 1: Implement Maximum Drawdown (10 minutes)

In [ ]:
# CHALLENGE: Implement maximum drawdown calculation

def calculate_max_drawdown(returns):
    """
    Calculate maximum drawdown from returns
    
    Parameters:
    -----------
    returns : array-like
        Asset returns
        
    Returns:
    --------
    float : Maximum drawdown (negative percentage)
    
    YOUR CODE HERE:
    """
    # Step 1: Calculate cumulative returns (growth of $1)
    # cumulative = ...
    
    # Step 2: Track running peak
    # peak = ...
    
    # Step 3: Calculate drawdown at each point
    # drawdown = ...
    
    # Step 4: Return minimum (maximum drawdown)
    # return ...
    pass

# TEST YOUR IMPLEMENTATION:
# Expected: max_dd should be negative
# max_dd = calculate_max_drawdown(returns_data)
# print(f"Max Drawdown: {max_dd:.2%}")

In [ ]:
# SOLUTION:
def calculate_max_drawdown_solution(returns):
    cumulative = np.cumprod(1 + returns)
    peak = np.maximum.accumulate(cumulative)
    drawdown = (cumulative - peak) / peak
    return np.min(drawdown)

max_dd = calculate_max_drawdown_solution(returns_data)
print(f"Max Drawdown: {max_dd:.2%}")

### Challenge 2: Walk-Forward Cross-Validation (15 minutes)

In [ ]:
# CHALLENGE: Implement walk-forward CV

def walk_forward_cv(X, y, model, train_size, test_size, gap=0):
    """
    Walk-forward cross-validation
    
    Parameters:
    -----------
    X : array (n_samples, n_features)
    y : array (n_samples,)
    model : sklearn model
    train_size : int
    test_size : int
    gap : int (embargo period)
    
    Returns:
    --------
    dict with predictions, actuals, and metrics
    
    YOUR CODE HERE:
    """
    predictions = []
    actuals = []
    
    # Start from after first training window
    # ...
    
    # Loop until we run out of data
    # while ...:
    #     ...
    
    return {
        'predictions': np.array(predictions),
        'actuals': np.array(actuals)
    }

# TEST:
# X_test = np.column_stack([np.roll(returns_data, i) for i in [1,5,10]])[20:]
# y_test = returns_data[20:]
# results = walk_forward_cv(X_test, y_test, Ridge(alpha=1.0), 252, 21, gap=1)
# print(f"Predictions: {len(results['predictions'])}")

In [ ]:
# SOLUTION:
def walk_forward_cv_solution(X, y, model, train_size, test_size, gap=0):
    predictions = []
    actuals = []
    
    n = len(X)
    start = train_size
    
    while start + gap + test_size <= n:
        # Split
        train_end = start
        test_start = start + gap
        test_end = test_start + test_size
        
        X_train, y_train = X[:train_end], y[:train_end]
        X_test, y_test = X[test_start:test_end], y[test_start:test_end]
        
        # Train and predict
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        
        predictions.extend(preds)
        actuals.extend(y_test)
        
        start += test_size
    
    return {
        'predictions': np.array(predictions),
        'actuals': np.array(actuals)
    }

# Test
X_test = np.column_stack([np.roll(returns_data, i) for i in [1,5,10]])[20:]
y_test = returns_data[20:]
results = walk_forward_cv_solution(X_test, y_test, Ridge(alpha=1.0), 252, 21, gap=1)
print(f"✅ Walk-forward predictions: {len(results['predictions'])}")

### Challenge 3: Break-Even Cost Analysis (10 minutes)

In [ ]:
# CHALLENGE: Find break-even transaction cost

def find_breakeven_cost(signals, returns, target_sharpe=0.0):
    """
    Find the cost level where strategy Sharpe equals target
    
    Parameters:
    -----------
    signals : array
    returns : array
    target_sharpe : float
    
    Returns:
    --------
    int : Break-even cost in basis points
    
    YOUR CODE HERE:
    """
    # Binary search or linear scan for break-even point
    # ...
    pass

# TEST:
# breakeven = find_breakeven_cost(signal, returns_data, target_sharpe=0.5)
# print(f"Break-even at {breakeven} bps for Sharpe = 0.5")

In [ ]:
# SOLUTION:
def find_breakeven_cost_solution(signals, returns, target_sharpe=0.0):
    for cost in range(0, 200):
        result = backtest_with_costs(signals, returns, cost)
        sharpe = np.mean(result['net']) / np.std(result['net']) * np.sqrt(252)
        if sharpe < target_sharpe:
            return cost - 1
    return 200

breakeven_0 = find_breakeven_cost_solution(signal, returns_data, target_sharpe=0.0)
breakeven_05 = find_breakeven_cost_solution(signal, returns_data, target_sharpe=0.5)
print(f"✅ Break-even (Sharpe=0): {breakeven_0} bps")
print(f"✅ Break-even (Sharpe=0.5): {breakeven_05} bps")

## 4. Behavioral Interview Questions

### Q1: "Walk me through how you would backtest a trading strategy."

**Sample Answer:**
1. **Data preparation**: Load point-in-time data, handle survivorship bias
2. **Feature engineering**: Create predictive features (momentum, volatility, etc.)
3. **Model training**: Use walk-forward validation with embargo
4. **Signal generation**: Convert predictions to trading signals
5. **Execution simulation**: Include transaction costs (commission + spread + slippage)
6. **Performance analysis**: Calculate Sharpe, max drawdown, sanity checks
7. **Validation**: Check for overfitting, run sensitivity analysis

### Q2: "Tell me about a time you discovered a bug in a backtest."

**Framework:**
- Situation: Describe the backtest and the suspiciously good results
- Task: What made you suspicious (Sharpe too high, etc.)
- Action: How you debugged (checked for lookahead, costs, etc.)
- Result: What you found and fixed

### Q3: "How would you explain overfitting to a non-technical stakeholder?"

**Sample Answer:**
"Imagine memorizing answers to a practice test instead of learning the material. You'd ace that specific test but fail a new one. Overfitting is similar - the model memorizes historical patterns but can't generalize to new data."

## 5. Quick Formula Reference

| Metric | Formula |
|--------|---------|
| Sharpe | $(R - R_f) / \sigma × \sqrt{252}$ |
| Sortino | $(R - R_f) / \sigma_{down} × \sqrt{252}$ |
| Max DD | $\max_t (Peak_t - Value_t) / Peak_t$ |
| Calmar | $R_{annual} / |MDD|$ |
| IR | $(R - R_{bench}) / \sigma_{active} × \sqrt{252}$ |

In [ ]:
# Quick reference implementations
def quick_metrics(returns):
    """Calculate all key metrics at once"""
    ret = pd.Series(returns)
    
    metrics = {
        'Sharpe': ret.mean() / ret.std() * np.sqrt(252),
        'Sortino': ret.mean() / ret[ret < 0].std() * np.sqrt(252),
        'Annual Return': ret.mean() * 252,
        'Annual Vol': ret.std() * np.sqrt(252),
    }
    
    # Max DD
    cum = (1 + ret).cumprod()
    peak = cum.expanding().max()
    dd = (cum - peak) / peak
    metrics['Max DD'] = dd.min()
    
    # Calmar
    metrics['Calmar'] = metrics['Annual Return'] / abs(metrics['Max DD'])
    
    return metrics

# Print reference
metrics = quick_metrics(returns_data)
print("📊 QUICK METRICS REFERENCE:")
for k, v in metrics.items():
    if 'Ratio' in k or k in ['Sharpe', 'Sortino', 'Calmar']:
        print(f"   {k}: {v:.2f}")
    else:
        print(f"   {k}: {v:.2%}")

## 6. Week 12 Checklist

Before moving to Week 13, ensure you can:

- [ ] Calculate Sharpe, Sortino, Max Drawdown from scratch
- [ ] Explain why K-Fold CV fails for time series
- [ ] Implement walk-forward validation
- [ ] Model transaction costs in backtests
- [ ] Identify lookahead bias and survivorship bias
- [ ] Explain overfitting in financial ML
- [ ] Apply multiple testing corrections
- [ ] Build a production-ready backtest pipeline

---

## 7. Key Takeaways

1. **Metrics**: Know Sharpe, Sortino, Max DD and their limitations
2. **Costs**: Include realistic transaction costs (5-20 bps typical)
3. **Validation**: Always use walk-forward, never K-Fold
4. **Bias**: Watch for lookahead, survivorship, overfitting
5. **Production**: Point-in-time data, sanity checks, monitoring

---

**Congratulations! You've completed Week 12!**

*Next: Week 13 - Neural Networks & Deep Learning*